In [ ]:
# Modules
from scipy import ndimage
import numpy as np
from PIL import Image, ImageFilter, ImageOps, ImageChops, ImageEnhance, ImageStat
from __future__ import print_function
import glob

from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import re

import sys
import datetime
import random
import warnings


# Functions
sys.path.insert(0, './helpers/')
from helpers import *
from model_unet import *

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed
np.random.seed(1)

In [ ]:
# Loads all data images in a list
filelist = glob.glob("../randomized_data/train/data_*.tiff")

# Sort the list according the their names 
# so that the the image and labels are on 
# the same position

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [ atoi(c) for c in re.split('(\d+)', text) ]

filelist.sort(key=natural_keys)

data =   [np.array(Image.open(fname).resize((256,256)), dtype=int) for fname in filelist if fname[-9:-5] != 'mask']

labels = [np.array(Image.open(fname).resize((256,256)), dtype=int) for fname in filelist if fname[-9:-5] == 'mask']

for lab in labels:
    
    lab[lab == 0] = 1
    lab[lab == 255] = 0
    
filelist = glob.glob("../randomized_data/test/data_*.tiff")

filelist.sort(key=natural_keys)

data_test =   [np.array(Image.open(fname).resize((256,256)), dtype=int) for fname in filelist if fname[-9:-5] != 'mask']

labels_test = [np.array(Image.open(fname).resize((256,256)), dtype=int) for fname in filelist if fname[-9:-5] == 'mask']

for lab in labels_test:
    
    lab[lab == 0] = 1
    lab[lab == 255] = 0

In [ ]:
labels = np.array(labels, dtype=int)
data = np.array(data, dtype=int)
labels_test = np.array(labels_test, dtype=int)
data_test = np.array(data_test, dtype=int)

labels = np.reshape(labels, (1888,256,256,1))
data = np.reshape(data, (1888,256,256,1))
labels_test = np.reshape(labels_test, (480,256,256,1))
data_test = np.reshape(data_test, (480,256,256,1))
print(labels_test.shape)
print(labels.shape)
print(data.shape)
print(data_test.shape)

In [ ]:
model = model_unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit(data, labels, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])